In [ ]:
from dataclasses import dataclass
from decimal import Decimal
from typing import Literal

from jetblack_pnl.core import (
    add_trade,
    TradingPnl,
    IMatchedPool,
    IUnmatchedPool,
    SplitTrade    
)
from jetblack_pnl.impl.simple import MatchedPool, UnmatchedPool, Security

class TradeData:

    def __init__(self, trade_id: int, description: str) -> None:
        self.trade_id = trade_id
        self.description = description

    def __repr__(self) -> str:
        return f"[{self.trade_id}] {self.description}"
    
Side = Literal['buy', 'sell']

def trade(trade_id: int, side: Side, quantity: int, price: int) -> SplitTrade[str]:
    sign = 1 if side == 'buy' else -1
    return Trade